In [53]:
import requests
import json
import time

In [8]:
API_key = 'PlIE5PSJaLBOTy67DORlQwY3MchgQxfP'
API_url = "https://eth-mainnet.g.alchemy.com/v2/PlIE5PSJaLBOTy67DORlQwY3MchgQxfP"
API_nft_url = 'https://eth-mainnet.g.alchemy.com/nft/v2/PlIE5PSJaLBOTy67DORlQwY3MchgQxfP'

In [59]:
# Global Data Structures
famous_NFT_collections = ['0x60E4d786628Fea6478F785A6d7e704777c86a7c6', '0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D', '0x34d85c9CDeB23FA97cb08333b511ac86E1C4E258', '0xED5AF388653567Af2F388E6224dC7C4b3241C544', '0x49cF6f5d44E70224e2E23fDcdd2C053F30aDA28B', '0x23581767a106ae21c074b2276D25e5C3e136a68b', '0xb7F7F6C52F2e2fdb1963Eab30438024864c313F6', '0x8a90CAb2b38dba80c64b7734e58Ee1dB38B8992e', '0xba30E5F9Bb24caa003E9f2f0497Ad287FDF95623', '0x7Bd29408f11D2bFC23c34f18275bBf23bB716Bc7']
# MutantApe, BoredApe, TheOtherside, AZUKI, CloneXToken, MoonBirds, CrpytoPunk, Doodle, BAKC, Meebits
print(len(famous_NFT_collections))
all_wallet_address = {}

10


In [60]:
def getNFTsForCollection(contractAddress, withMetadata, limit, lastTokenid):
  if lastTokenid == -1:
    url = API_url + '/getNFTsForCollection?contractAddress=' + contractAddress + '&withMetadata=' + str(withMetadata) + '&limit=' + str(limit)
    r = requests.get(url, headers={"accept": "application/json"})
    return r
  else:
    url = API_url + '/getNFTsForCollection?contractAddress=' + contractAddress + '&withMetadata=' + str(withMetadata) + '&limit=' + str(limit) + '&startToken=' + str(lastTokenid)
    r = requests.get(url, headers={"accept": "application/json"})
    return r

def getNFTSales(contractAddress, tokenid):
	# from block 0 to latest block, with ascending order
	url = API_url + '/getNFTSales?fromBlock=0&toBlock=latest&order=asc&contractAddress=' + contractAddress + '&tokenId=' + str(tokenid)
	r = requests.get(url, headers={"accept": "application/json"})
	return r

In [67]:
def getAllNFTs(contractAddress):
  allNFTs = []
  lastTokenid = -1
  while True:
    time.sleep(0.1)
    r = getNFTsForCollection(contractAddress=contractAddress, withMetadata=False, limit=100, lastTokenid=lastTokenid).text
    newNFT = json.loads(r)['nfts']
    if 'nextToken' not in json.loads(r).keys():
      break
    allNFTs.extend(newNFT)
    break
    lastTokenid = json.loads(r)['nextToken']
  return allNFTs

In [64]:
# given a NFT and its token id, get its owners' history
def OwnerHistory(contractAddress, tokenid):
	time.sleep(0.1)
	r = getNFTSales(contractAddress, tokenid).text
	xaction = json.loads(r)
	for history in xaction['nftSales']:
		#print(history['buyerAddress'], history['sellerAddress'])
		all_wallet_address[history['buyerAddress']] = 1
		all_wallet_address[history['sellerAddress']] = 1

In [ ]:
for collection in famous_NFT_collections:
  nfts = getAllNFTs(collection)
  for nft in nfts:
    if 'id' in nft.keys() and 'tokenId' in nft['id'].keys():
      OwnerHistory(collection, nft['id']['tokenId'])

with open("all_wallet_address.json", "w") as fp:
    json.dump(all_wallet_address,fp)